<a href="https://colab.research.google.com/github/GouthamVicky/ResearchPaperSummarization/blob/main/ContribSenEvaluvation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Model Evaluation on https://aclanthology.org/P19-1106/**

We will use Grobid to extract Raw text from the PDF and pass the text to stanza which provides sentence level breakdown

Sentences will be passed to the classification pipeline predicts whether the given sentences are contribution sentences or not 


The concatenation Predicted contributed sentences will be considered as generated summary of the paper

The Generated summary will be compared with the abstract of the paper to calculate the metrics

In [1]:
!pip install transformers Stanza requests urllib3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=70bed2267110829620b653a2282be6c14aed65d95908561402aaf9caef9eaae8
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


### **Download the Evaluvation Paper from https://aclanthology.org/P19-1106/**

In [2]:
!wget https://aclanthology.org/P19-1106.pdf

--2023-03-04 14:58:12--  https://aclanthology.org/P19-1106.pdf
Resolving aclanthology.org (aclanthology.org)... 174.138.37.75
Connecting to aclanthology.org (aclanthology.org)|174.138.37.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 942140 (920K) [application/pdf]
Saving to: ‘P19-1106.pdf’

P19-1106.pdf        100%[===================>] 920.06K  5.30MB/s    in 0.2s    

2023-03-04 14:58:13 (5.30 MB/s) - ‘P19-1106.pdf’ saved [942140/942140]



### **Extract Text from the paper using Grobid Service API**

To Learn more info about Grobid Refer : [Grobid Github](https://github.com/kermitt2/grobid)

*Note - This might takes few mins to extract text since we are using Grobid service API hosted in huggingface space, Extraction latency depends upon the traffic load at the time*

In [3]:
import requests
pdf_file = open('/content/P19-1106.pdf', 'rb')

files = {'input': pdf_file}

response = requests.post('https://kermitt2-grobid-crf.hf.space/api/processFulltextDocument', files=files)

### **To Extract only the abstract from the given paper using beautiful Soup**

In [4]:
from bs4 import BeautifulSoup
# # Parse the TEI XML response using BeautifulSoup 
soup = BeautifulSoup(response.text, "xml")

# find the abstract element
abstract_element = soup.find('abstract')

# extract the text content of the abstract element
abstract_text = abstract_element.text.strip()

print("Abstract "+"\n\n"+abstract_text)

Abstract 

Automatically validating a research artefact is one of the frontiers in Artificial Intelligence (AI) that directly brings it close to competing with human intellect and intuition. Although criticized sometimes, the existing peer review system still stands as the benchmark of research validation. The present-day peer review process is not straightforward and demands profound domain knowledge, expertise, and intelligence of human reviewer(s), which is somewhat elusive with the current state of AI. However, the peer review texts, which contains rich sentiment information of the reviewer, reflecting his/her overall attitude towards the research in the paper, could be a valuable entity to predict the acceptance or rejection of the manuscript under consideration. Here in this work, we investigate the role of reviewers sentiments embedded within peer review texts to predict the peer review outcome. Our proposed deep neural architecture takes into account three channels of informati

### **Extract Paragraph from the Grobid Ouput response and concat it with the abstract Text**

In [5]:
paragraphs_list=[]
#Find the paragrph text inside <p> element
paragraphs = soup.find_all("p")

for paragraph in paragraphs:
    paragraphs_list.append(paragraph.get_text())

#Concat the abstract and paragraph text
output =  str(abstract_text) + ("\n\n").join(paragraphs_list) 

In [6]:
print(output)

Automatically validating a research artefact is one of the frontiers in Artificial Intelligence (AI) that directly brings it close to competing with human intellect and intuition. Although criticized sometimes, the existing peer review system still stands as the benchmark of research validation. The present-day peer review process is not straightforward and demands profound domain knowledge, expertise, and intelligence of human reviewer(s), which is somewhat elusive with the current state of AI. However, the peer review texts, which contains rich sentiment information of the reviewer, reflecting his/her overall attitude towards the research in the paper, could be a valuable entity to predict the acceptance or rejection of the manuscript under consideration. Here in this work, we investigate the role of reviewers sentiments embedded within peer review texts to predict the peer review outcome. Our proposed deep neural architecture takes into account three channels of information: the pap

### **Pass the Extracted text to stanza to break down into structured sentences**

In [7]:
import stanza
nlp = stanza.Pipeline('en')
doc = nlp(output)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: constituency
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [8]:
sentences = [' '.join([token.text for token in sentence.tokens]) for sentence in doc.sentences]

### **Load the Trained model to classify contribution sentenes**

In [9]:
from transformers import pipeline
from transformers import BertTokenizer, BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("Goutham-Vignesh/ContributionSentClassification-scibert")

tokenizer=BertTokenizer.from_pretrained('Goutham-Vignesh/ContributionSentClassification-scibert')
text_classification = pipeline('text-classification', model=model, tokenizer=tokenizer)

### **Model Prediction and Summary Generation**

Pass the output sentences list from stanza to the model pipeline to predict the contribution sentences and concatenate those sentences to generate summary

In [10]:
generated_summary=""
for line in sentences:
  prediction = text_classification(line)
  if prediction[0]['label']=="LABEL_1":
    generated_summary=generated_summary+line

print( "GENERATED SUMMARY ")
print(generated_summary)

GENERATED SUMMARY 
Automatically validating a research artefact is one of the frontiers in Artificial Intelligence ( AI ) that directly brings it close to competing with human intellect and intuition .Automatically validating a research artefact is one of the frontiers in Artificial Intelligence ( AI ) that directly brings it close to competing with human intellect and intuition .The evaluation shows that our proposed model successfully outperforms the earlier reported results in PeerRead .Finally , we fuse the extracted review sentiment feature and joint paper + review representation together to generate the overall recommendation score ( Decision - Level Fusion ) using the affine transformation asWe employ a grid search for hyperparameter optimization .For Task 1 , F is 256 , l is 5 . ReLU is the non-linear function g( ) , learning rate is 0.007 .We train the model with SGD optimizer , set momentum as 0.9 and batch size as 32 .We keep dropout at 0.5 .Again we train the model with Ada

### **Compute the Metrics by comparing with the abstract of the paper**

In [20]:
!pip install evaluate bert-score rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=d580f589d0a5b00a7e40b8ac9da795a758e6981f3c4a47c566703bdac7b8da3b
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score


In [21]:
import evaluate
from evaluate import load
bertscore = evaluate.load("bertscore")
rouge = evaluate.load('rouge')


### **Rouge Score Metrics**

In [22]:
results = rouge.compute(predictions=[generated_summary], references=[abstract_text])
print(results)

{'rouge1': 0.38137472283813745, 'rouge2': 0.14699331848552338, 'rougeL': 0.24390243902439024, 'rougeLsum': 0.24390243902439024}


### **Bert Score Metrics**

In [19]:
result = bertscore.compute(predictions=[generated_summary], references=[abstract_text], lang="en")
metricsResult={"precision":result['precision'],"recall":result['recall'],"f1":result['f1']}
print(metricsResult)

{'precision': [0.8174327611923218], 'recall': [0.8349176645278931], 'f1': [0.826082706451416]}
